In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Sample data
ratings_dict = {
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1],
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3],
}

df = pd.DataFrame(ratings_dict)

# Pivot the data to create a user-item matrix
user_item_matrix = df.pivot(index='user', columns='item', values='rating')

# Normalize the user-item matrix
user_item_matrix_norm = user_item_matrix.subtract(user_item_matrix.mean(axis=1), axis=0)

# Calculate cosine similarity
user_similarity = cosine_similarity(user_item_matrix_norm.fillna(0))
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def predict_rating(user_id, item_id, user_item_matrix, user_similarity_df):
    if item_id not in user_item_matrix.columns:
        return np.nan

    similar_users = user_similarity_df[user_id].drop(user_id).dropna()
    similar_users = similar_users[similar_users > 0]

    if similar_users.empty:
        return np.nan

    similar_user_ratings = user_item_matrix.loc[similar_users.index, item_id]
    weighted_sum = np.dot(similar_users, similar_user_ratings.fillna(0))
    similarity_sum = similar_users.sum()

    if similarity_sum == 0:
        return np.nan

    return weighted_sum / similarity_sum

# Predict ratings for the test set
df['predicted_rating'] = df.apply(lambda row: predict_rating(row['user'], row['item'], user_item_matrix, user_similarity_df), axis=1)

# Example prediction
user_id = 'A'
item_id = 1
pred = predict_rating(user_id, item_id, user_item_matrix, user_similarity_df)
print(f"Predicted rating for user {user_id} and item {item_id}: {pred:.2f}")

# Calculate RMSE
testset = df.dropna(subset=['predicted_rating'])
rmse = np.sqrt(((testset['rating'] - testset['predicted_rating']) ** 2).mean())
print(f'RMSE: {rmse:.2f}')


Predicted rating for user A and item 1: 3.00
RMSE: 1.58
